# Lab 06: Spatial smoothing, denoising with ICA, and temporal filtering

The goal of this lab is to practice common pre-preprocessing steps that all involve some form of filtering out signal that is deemed to be "noise" based on signal processing concepts. We will look at three common steps that follow motion correction and slice-timing: 
* Spatial smoothing
* ICA-based denoising
* Temporal filtering

## Preparation

In [ ]:
%%bash

# DECIDE WHETHER YOU WANT TO COPY (cp) OR DOWNLOAD THE FILES NECESSARY FOR THIS LAB

# default
ans="dl"

if [ "${ans}" == "cp" ]; then
    cp ../05-Lab/my_bold_mcf_st.nii.gz my_bold_mcf_st.nii.gz
    cp ../05-Lab/my_bold_mcf.par my_bold_mcf.par
    cp ../04-Lab/func2T1.mat func2T1.mat
    cp ../04-Lab/T1_2_MNI_warp.nii.gz T1_2_MNI_warp.nii.gz
    # need some other generated outputs from ICA-AROMA
elif [ "${ans}" == "dl" ]; then
    wget --quiet -O 06-Lab_data.tar.gz https://osf.io/kf43u/download &&\
    tar -xf 06-Lab_data.tar.gz &&\
    rm 06-Lab_data.tar.gz
else
    printf 'YOU FORGOT TO CHANGE ANS TO "cp" OR "dl"'
fi

## Spatial smoothing in FSL

FSL's smoothing tool is called SUSAN
* See user manual [here](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/SUSAN)
* An important usage note is that SUSAN takes in the smoothing kernel as "sigma" which refers to the standard deviation of the smoothing kernel rather than the Full-Width at Half-Max (FWHM) in mm. 
    * [Link](https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=FSL;d7249c17.1301) to an informative post on the FSL listserv
    * Conversion from sigma to FWHM = `sigma*sqrt(8*ln(2)) = sigma*2.3548`
    * Conversion from FWHM to sigma = `mm/sqrt(8*ln(2))`

In [ ]:
# Usage 
susan

In [ ]:
# Strip the skull from the motion corrected and possibly slice time corrected image

# Take mean of the functional image over time
fslmaths my_bold_mcf_st.nii.gz -Tmean mean_func
# Bet the mean func, tell it to only generate the mask for now, then we rename mask_mask to mask
bet2 mean_func mask -f .3 -n -m; mv mask_mask.nii.gz mask.nii.gz
# Apply mask to create brain extracted functional image
fslmaths my_bold_mcf_st.nii.gz -mas mask.nii.gz my_bold_mcf_st_bet.nii.gz

In [ ]:
# Take a look
fslview my_bold_mcf_st_bet.nii.gz mask.nii.gz

In [ ]:
# FSL preprocessing steps to find brightness thresh and create liberal functional mask 
robust_range=$(fslstats my_bold_mcf_st_bet.nii.gz -p 2 -p 98)

# Find "robust" range from 2nd to 98th percentile
# Take 10th percentile of this robust range and threshold functional image and make new mask
mask_thres=$(echo "scale=1; var1=$(echo ${robust_range} | awk -F' ' '{print $2}'); var2=var1/10; var2" | bc)

# Voxels most likely noise get excluded from this mask
fslmaths my_bold_mcf_st_bet.nii.gz -thr ${mask_thres} -Tmin -bin mask -odt char

# Dilate the mask to give some extra buffer space
fslmaths mask -dilF mask

# Find the median within this mask and then 75% of the median within that mask 
brightness_thresh=$(fslstats my_bold_mcf_st.nii.gz -k mask -p 50 | awk '{print ($1*0.75)}')

# Apply mask and create new mean func
fslmaths my_bold_mcf_st.nii.gz -mas mask.nii.gz my_bold_mcf_st_thresh.nii.gz
fslmaths my_bold_mcf_st_thresh.nii.gz -Tmean mean_func


In [ ]:
# Now we're ready to smooth!

# If working from the terminal, convert desired FWHM (mm) to sigma for input to SUSAN
# Below is an example of smoothing at 6mm FWHM
smooth=6
smoothSigma=$(echo $smooth | awk '{print ($1/(sqrt(8*log(2))))}')
echo "Smoothing kernel set to ${smooth}mm which is ${smoothSigma} in sigma"

In [ ]:
# Call to SUSAN
susan my_bold_mcf_st_thresh.nii.gz \
      ${brightness_thresh} ${smoothSigma} \
      3 1 1 \
      mean_func.nii.gz \
      ${brightness_thresh} \
      my_bold_mcf_st_thresh_smooth-6mm.nii.gz


In [ ]:
# Compare un-smoothed and smoothed functional image
fslview my_bold_mcf_st_thresh.nii.gz my_bold_mcf_st_thresh_smooth.nii.gz

## ICA-AROMA

* It's my understanding that median intensity scaling isn't necessary so we could simply use as inputs:
    * my_bold_mcf_st_thresh_smooth.nii.gz
    * .par file copied above for motion parameters
    * reg files copied above
    * create a more snug mask with our mean func as suggested in manual
* Should we have them download ica-aroma from github and run or just download output and understand? Any preference on clone vs. download when pulling a package from github that you won't contributed to. Cloning can make it easier to pull in updates?
* It would take too long to run during class and to run on a new data set could be their lab problem since they'd have to run from scratch

In [ ]:
git clone https://github.com/maartenmennes/ICA-AROMA.git

In [ ]:
# Make a snugger mask
bet mean_func.nii.gz aroma_mask -f .3 -n -m

In [ ]:
fslview mean_func.nii.gz aroma_mask_mask.nii.gz

In [ ]:
# Call to ICA-AROMA

# ICA-AROMA/ICA_AROMA.py \
#   -in ../my_bold_mcf_st_thresh_smooth-6mm.nii.gz \
#   -out ../ica_aroma_6mm \
#   -affmat ../func2T1.mat \
#   -warp ../T1_2_MNI_warp.nii.gz \
#   -mc ../my_bold_mcf.par \
#   -m ../aroma_mask_mask.nii.gz

In [ ]:
hp_freq=0.008
lp_freq=0.08
hp_sigma=$(echo "1/(2*${TR}*${hp_freq})" | bc -l)
lp_sigma=$(echo "1/(2*${TR}*${lp_freq})" | bc -l)


In [ ]:
fslmaths | grep -- "-bptf"

In [ ]:
fslmaths ica_aroma_6mm/denoised_func_data_nonaggr.nii.gz \
         -bptf ${hp_sigma} ${lp_sigma} \
         my_bold_mcf_st_thresh_smooth-6mm_tempfilt-fsl.nii.gz

In [ ]:
3dBandpass -prefix my_bold_mcf_st_thresh_smooth-6mm_tempfilt-afni.nii.gz \
           ${hp_freq} ${lp_freq} \
           ica_aroma_6mm/denoised_func_data_nonaggr.nii.gz

In [ ]:
# FSL power spectrum
fslpspec my_bold_mcf_st_thresh_smooth-6mm_tempfilt-fsl.nii.gz pwr_spct_fsl.nii.gz
# get a summary measure
fslmeants -i pwr_spct_fsl.nii.gz -m aroma_mask_mask.nii.gz

# AFNI power spectrum
fslpspec my_bold_mcf_st_thresh_smooth-6mm_tempfilt-afni.nii.gz pwr_spct_afni.nii.gz
# get a summary measure
fslmeants -i pwr_spct_afni.nii.gz -m aroma_mask_mask.nii.gz